**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-imor74wb
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-imor74wb
  Resolved https://github.com/Kaggle/learntools.git to commit 010e3b5035354e15c073a0aca9e202c2e2beb742
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=269001 sha256=419be6f6733bf6be2411014591b5c5095c40f97d3472d302f6221c6f7809b8b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-3s4lguai/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5003bdd51918a9b5a52134096663b4d7e02395c5,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-02-25 14:15:00+00:00,2013-02-25 14:15:00+00:00,120,0.00,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,720534d264001b2644f682755b294067fdf1da21,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-02 08:15:00+00:00,2013-06-02 08:15:00+00:00,60,0.02,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,515b9a6d5234a4d1fb559dca27b3e1cc541a342d,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-24 06:15:00+00:00,2013-06-24 06:15:00+00:00,60,0.04,<NA>,<NA>,1,77,...,NaN,NaN,Cash,None,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)
3,8afd48d966bf464e93127f43b72a8e405596a8eb,0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...,2013-02-01 07:00:00+00:00,2013-02-02 00:00:00+00:00,61560,0.00,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,Cash,None,NaN,NaN,None,NaN,NaN,None
4,fe73a44f01fdd2a3c740a01f53e97ec2ced93455,0150188f8c8e8973a198d4af0b427fca8ff48df2662d6b...,2013-02-01 07:00:00+00:00,2013-02-02 00:00:00+00:00,60900,0.00,<NA>,<NA>,<NA>,<NA>,...,0.0,0.0,Cash,None,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
avg_num_trips_query = """
    WITH trips_by_day AS
    (
    SELECT DATE(trip_start_timestamp) AS trip_date,
        COUNT(*) AS num_trips
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE trip_start_timestamp >= '2016-01-01' AND 
        trip_start_timestamp < '2016-04-01'
    GROUP BY trip_date
    ORDER BY trip_date
    )
    SELECT trip_date, num_trips, 
        AVG(num_trips)
        OVER (
            ORDER BY trip_date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
        ) AS avg_num_trips
    FROM trips_by_day
    ORDER BY trip_date
"""

client.query(avg_num_trips_query).result().to_dataframe()

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,num_trips,avg_num_trips
0,2016-01-01,97202,73313.750000
1,2016-01-02,69000,72998.200000
2,2016-01-03,59286,73452.166667
3,2016-01-04,67767,74401.428571
4,2016-01-05,71736,73767.285714
...,...,...,...
86,2016-03-27,65311,84715.000000
87,2016-03-28,73250,78639.428571
88,2016-03-29,78059,75684.000000
89,2016-03-30,89492,74274.400000


In [4]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,avg_num_trips
0,2016-02-23,92244.285714
1,2016-02-16,90901.142857
2,2016-01-29,79909.857143
3,2016-02-22,90943.428571
4,2016-01-07,75520.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [5]:
# Lines below will give you a hint or solution code
#q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [6]:
# Easy look.
client.list_rows(table, max_results=3).to_dataframe()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5003bdd51918a9b5a52134096663b4d7e02395c5,2130bc5fd239a4e3b304662424fb4cc7db0ca7abf78cc5...,2013-02-25 14:15:00+00:00,2013-02-25 14:15:00+00:00,120,0.00,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,720534d264001b2644f682755b294067fdf1da21,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-02 08:15:00+00:00,2013-06-02 08:15:00+00:00,60,0.02,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,515b9a6d5234a4d1fb559dca27b3e1cc541a342d,4bb55b69e710d1792f5fd4888001f4ff0ce34040f976f8...,2013-06-24 06:15:00+00:00,2013-06-24 06:15:00+00:00,60,0.04,<NA>,<NA>,1,77,...,NaN,NaN,Cash,None,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)


In [7]:
trip_number_query = """
    SELECT pickup_community_area, trip_start_timestamp, trip_end_timestamp,
        RANK() OVER
        (PARTITION BY pickup_community_area ORDER BY trip_start_timestamp) AS trip_number
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE DATE(trip_start_timestamp) = '2013-10-03'    
"""

client.query(trip_number_query).result().to_dataframe()

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,4,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,4,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,4,2013-10-03 00:15:00+00:00,2013-10-03 00:30:00+00:00,2
3,4,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
4,4,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
...,...,...,...,...
84263,8,2013-10-03 23:45:00+00:00,2013-10-04 00:00:00+00:00,23609
84264,8,2013-10-03 23:45:00+00:00,2013-10-04 00:00:00+00:00,23609
84265,8,2013-10-03 23:45:00+00:00,2013-10-04 00:00:00+00:00,23609
84266,8,2013-10-03 23:45:00+00:00,2013-10-04 00:00:00+00:00,23609


In [8]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER(
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                            ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,70,2013-10-03 01:15:00+00:00,2013-10-03 01:30:00+00:00,1
1,70,2013-10-03 07:45:00+00:00,2013-10-03 08:00:00+00:00,2
2,70,2013-10-03 08:00:00+00:00,2013-10-03 08:45:00+00:00,3
3,70,2013-10-03 13:30:00+00:00,2013-10-03 14:00:00+00:00,4
4,70,2013-10-03 19:30:00+00:00,2013-10-03 19:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [10]:
# for diff --> initial, (lagged) end, then time unit 
# done some editing to make sure non-negative result.
break_time_query = """
    SELECT taxi_id, 
        trip_start_timestamp, 
        trip_end_timestamp,
        TIMESTAMP_DIFF (
            trip_start_timestamp, 
            LAG(trip_end_timestamp) 
                OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp, trip_end_timestamp),
            MINUTE) AS prev_break
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
    ORDER BY taxi_id
"""

client.query(break_time_query).result().to_dataframe()[:10]

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:15:00+00:00,2013-10-03 11:30:00+00:00,<NA>
1,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:30:00+00:00,2013-10-03 11:45:00+00:00,0
2,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 11:45:00+00:00,2013-10-03 12:00:00+00:00,0
3,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:00:00+00:00,2013-10-03 12:00:00+00:00,0
4,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:15:00+00:00,2013-10-03 12:15:00+00:00,15
5,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:15:00+00:00,2013-10-03 12:30:00+00:00,0
6,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:30:00+00:00,2013-10-03 12:30:00+00:00,0
7,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 12:30:00+00:00,2013-10-03 12:45:00+00:00,0
8,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 13:00:00+00:00,2013-10-03 13:15:00+00:00,15
9,0008de7a146802839c9e6059f482d292ebdae13c5c31dd...,2013-10-03 13:30:00+00:00,2013-10-03 13:30:00+00:00,15


In [11]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,022bd6b4531df7aa8c253e9c9a459e59440cc0770a90db...,2013-10-03 15:30:00+00:00,2013-10-03 15:30:00+00:00,210
1,022bd6b4531df7aa8c253e9c9a459e59440cc0770a90db...,2013-10-03 19:15:00+00:00,2013-10-03 20:15:00+00:00,180
2,135ae3e27e645db0d93702c52df40e6f0ae8b88c87869b...,2013-10-03 17:15:00+00:00,2013-10-03 17:30:00+00:00,975
3,155ffe17bc32e7c3bfdfdba2750e82c6a23dab24f77d6c...,2013-10-03 22:15:00+00:00,2013-10-03 22:15:00+00:00,-60
4,2357184acaed8552382ea5d267f32c315561c998c0ffb2...,2013-10-03 11:30:00+00:00,2013-10-03 12:15:00+00:00,675


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*